In [3]:
# Hack to always autoreload modules and avoid restarting the kernel each time

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [4]:
# Note: This is a hack to allow importing from the parent directory
import sys
from pathlib import Path

sys.path.append(str(Path().resolve().parent))

In [5]:
import optuna
import torch
from models.gaussian_image_trainer import GaussianImageTrainer
from configs import Config
from utils.search_space import is_valid_combination

/home/h/harjesruiloba/miniforge3/envs/nerfstudio/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import torchvision
from constants import CIFAR10_TRANSFORM
from utils.data import create_default_image
from utils import visualize_tensor

# Only used one image for the first tuning
dataset = torchvision.datasets.CIFAR10(root="./data", download=True)
# cifar_image = CIFAR10_TRANSFORM(dataset[2][0])
cifar_imgs = [CIFAR10_TRANSFORM(dataset[i][0]) for i in range(4)]
visualize_tensor(cifar_imgs[1])

Files already downloaded and verified


In [7]:
import logging
import sys

In [8]:
# New experiments with some key learnings:]
# 1. LR 0.01 seems to be the best.
# 2. Max Steps 2000 is better than 1000, larger values are not much better.
# 3. Init Type: KNN is the best, Grid is still interesting for other reasons (AE), don't use random. 
# 4. Model Type: 2dgs is superior
# 5. More points is better, but 32x32 is already very good.
# 6. Init Scale 2.0 is clearly the best, bigger values might also be good but were not tested.
# 7. Init Opacity between 0.5 and 0.75 is the best.
# 8. Scale Regulation 0.1 is the best.
# 9. Opacity Regulation 0.5 is the best.
# 10. Extent 2 and 4 are similar, 1 is worse.

# Add stream handler of stdout to show the messages
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name_grid = "study-w-grid-2"  # Unique identifier of the study.
storage_name_grid = "sqlite:///{}.db".format(study_name_grid)
study_w_grid = optuna.create_study(study_name=study_name_grid, storage=storage_name_grid, load_if_exists=True)

Error: no "view" rule for type "image/png" passed its test case
       (for more information, add "--debug=1" on the command line)


[I 2025-01-09 00:04:39,804] Using an existing study with name 'study-w-grid-2' instead of creating a new one.


Using an existing study with name 'study-w-grid-2' instead of creating a new one.


In [9]:
def objective_w_grid_2(trial: optuna.Trial) -> float:
    """
    Objective function for Optuna hyperparameter optimization.

    Args:
        trial (optuna.Trial): The trial object used to suggest hyperparameters.

    Returns:
        float: The evaluation metric to minimize (e.g., validation loss).
    """
    # Conditional sampling
    # group_optimization = trial.suggest_categorical("group_optimization", [True, False])
    # strategy = None
    group_optimization = True # For now, as having default strategy was causing an error
    if not group_optimization:
        strategy = trial.suggest_categorical("strategy", ["default", "mcmc"])

    # Hyperparameter suggestions
    loss_weights = [
        1/3,
        1/3,
        1/3,
    ]

    scale_regularization = trial.suggest_categorical("scale_regularization", [None, 0.1, 0.5, 1.0])
    opacity_regularization = trial.suggest_categorical("opacity_regularization", [None, 0.1, 0.5, 1.0])

    # init_type = trial.suggest_categorical("init_type", ["grid", "knn"])
    init_type = "grid"  # Focus on the option with a fixed grid for this study
    num_points = 32*32
    extent = trial.suggest_categorical("extent", [1.0, 2.0, 4.0])
    init_scale = trial.suggest_categorical("init_scale", [2.0, 3.0, 4.0])
    init_opacity = trial.suggest_categorical("init_opacity", [0.5, 0.6, 0.7])

    max_steps = 1000
    learning_rate = 0.01

    model_type = "2dgs"  # Left out "2dgs-inria" as it needed further dependencies
    bilateral_grid = False

    image_index = trial.suggest_categorical("image_index", [0, 1, 2, 3])

    if not is_valid_combination({
        "learning_rate": learning_rate,
        "loss_weights": loss_weights,
        "group_optimization": group_optimization,
        # "strategy": strategy,
        "learning_rate": learning_rate,
        "model_type": model_type,
        "bilateral_grid": bilateral_grid,
    }, GaussianImageTrainer):
        raise optuna.exceptions.TrialPruned("Invalid hyperparameter combination")

    # Create Config object
    cfg = Config(
        save_results=False,
        save_logs=False,
        seed=42,
        image=cifar_imgs[image_index],  # Replace with actual ground truth image tensor
        max_steps=max_steps,
        learning_rate=learning_rate,
        loss_weights=loss_weights,
        init_type=init_type,
        num_points=num_points,
        init_scale=init_scale,
        init_opacity=init_opacity,
        # Fixed, added these:
        scale_regularization=scale_regularization,
        opacity_regularization=opacity_regularization,
        extent=extent,
        group_optimization=group_optimization,
        # strategy=strategy,
        model_type=model_type,
        # bilateral_grid=bilateral_grid,
        bilateral_grid=False
    )

    # Initialize and train the model
    trainer = GaussianImageTrainer(cfg)
    result = trainer.train()

    # Evaluate generated image quality (e.g., L1 loss with ground truth)
    generated_image = result.cpu()  # Replace with actual rendered output
    ground_truth_image = cfg.image.cpu()
    # evaluation_metric = torch.nn.functional.l1_loss(generated_image, ground_truth_image)  # Old version
    evaluation_metric = trainer.l1_loss * 1/3 + trainer.mse_loss * 1/3 + trainer.ssim_loss * 1/3  # Simply take average as loss

    return evaluation_metric.item()

In [18]:
study_w_grid.optimize(objective_w_grid_2, n_trials=1000, n_jobs=1, timeout=3600)

Model initialized. Number of Gaussians: 1024


/home/h/harjesruiloba/miniforge3/envs/nerfstudio/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `StructuralSimilarityIndexMeasure` from `torchmetrics` was deprecated and will be removed in 2.0. Import `StructuralSimilarityIndexMeasure` from `torchmetrics.image` instead.
  _future_warning(
/home/h/harjesruiloba/miniforge3/envs/nerfstudio/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `PeakSignalNoiseRatio` from `torchmetrics` was deprecated and will be removed in 2.0. Import `PeakSignalNoiseRatio` from `torchmetrics.image` instead.
  _future_warning(


Loss: 0.241 (L1: 0.061, MSE: 0.007, SSIM: 0.352): 100%|██████| 1000/1000 [00:09<00:00, 108.78step/s]
[I 2025-01-08 23:16:40,619] Trial 10 finished with value: 0.13969792425632477 and parameters: {'scale_regularization': 0.5, 'opacity_regularization': 1.0, 'extent': 1.0, 'init_scale': 2.0, 'init_opacity': 0.6, 'image_index': 0}. Best is trial 7 with value: 0.03648029640316963.


Final loss: 0.24079476296901703
Total Time: Rasterization: 1.187s, Backward: 3.891s
Trial 10 finished with value: 0.13969792425632477 and parameters: {'scale_regularization': 0.5, 'opacity_regularization': 1.0, 'extent': 1.0, 'init_scale': 2.0, 'init_opacity': 0.6, 'image_index': 0}. Best is trial 7 with value: 0.03648029640316963.
Trial 10 finished with value: 0.13969792425632477 and parameters: {'scale_regularization': 0.5, 'opacity_regularization': 1.0, 'extent': 1.0, 'init_scale': 2.0, 'init_opacity': 0.6, 'image_index': 0}. Best is trial 7 with value: 0.03648029640316963.
Model initialized. Number of Gaussians: 1024


Loss: 0.118 (L1: 0.041, MSE: 0.004, SSIM: 0.119): 100%|██████| 1000/1000 [00:08<00:00, 120.77step/s]
[I 2025-01-08 23:16:49,556] Trial 11 finished with value: 0.054165080189704895 and parameters: {'scale_regularization': 0.1, 'opacity_regularization': None, 'extent': 2.0, 'init_scale': 4.0, 'init_opacity': 0.7, 'image_index': 1}. Best is trial 7 with value: 0.03648029640316963.


...

Loss: 0.484 (L1: 0.188, MSE: 0.049, SSIM: 0.571):   7%|▌       | 71/1000 [00:00<00:06, 134.16step/s]
[W 2025-01-08 23:54:52,018] Trial 270 failed with parameters: {'scale_regularization': 0.1, 'opacity_regularization': None, 'extent': 4.0, 'init_scale': 2.0, 'init_opacity': 0.7, 'image_index': 2} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/h/harjesruiloba/miniforge3/envs/nerfstudio/lib/python3.8/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/tmp/user/22473/ipykernel_2012958/1870793656.py", line 80, in objective_w_grid_2
    result = trainer.train()
  File "/home/h/harjesruiloba/Projects/visual-representation-learning/models/gaussian_image_trainer.py", line 555, in train
    * torch.abs(torch.exp(self.splats["scales"])).mean()
KeyboardInterrupt


Trial 270 failed with parameters: {'scale_regularization': 0.1, 'opacity_regularization': None, 'extent': 4.0, 'init_scale': 2.0, 'init_opacity': 0.7, 'image_index': 2} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/h/harjesruiloba/miniforge3/envs/nerfstudio/lib/python3.8/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/tmp/user/22473/ipykernel_2012958/1870793656.py", line 80, in objective_w_grid_2
    result = trainer.train()
  File "/home/h/harjesruiloba/Projects/visual-representation-learning/models/gaussian_image_trainer.py", line 555, in train
    * torch.abs(torch.exp(self.splats["scales"])).mean()
KeyboardInterrupt
Trial 270 failed with parameters: {'scale_regularization': 0.1, 'opacity_regularization': None, 'extent': 4.0, 'init_scale': 2.0, 'init_opacity': 0.7, 'image_index': 2} because of the following error: KeyboardInterrupt().
Traceback (most recent call last)

[W 2025-01-08 23:54:52,021] Trial 270 failed with value None.


Trial 270 failed with value None.
Trial 270 failed with value None.


KeyboardInterrupt: 

In [19]:
study_w_grid.best_trial

FrozenTrial(number=151, state=TrialState.COMPLETE, values=[0.008646728470921516], datetime_start=datetime.datetime(2025, 1, 8, 23, 37, 4, 48667), datetime_complete=datetime.datetime(2025, 1, 8, 23, 37, 13, 201682), params={'scale_regularization': 0.1, 'opacity_regularization': None, 'extent': 4.0, 'init_scale': 2.0, 'init_opacity': 0.5, 'image_index': 2}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'scale_regularization': CategoricalDistribution(choices=(None, 0.1, 0.5, 1.0)), 'opacity_regularization': CategoricalDistribution(choices=(None, 0.1, 0.5, 1.0)), 'extent': CategoricalDistribution(choices=(1.0, 2.0, 4.0)), 'init_scale': CategoricalDistribution(choices=(2.0, 3.0, 4.0)), 'init_opacity': CategoricalDistribution(choices=(0.5, 0.6, 0.7)), 'image_index': CategoricalDistribution(choices=(0, 1, 2, 3))}, trial_id=152, value=None)

In [10]:
len(study_w_grid.trials)

328